In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py

from matplotlib import animation
from matplotlib import cm
from mpl_toolkits import mplot3d
import os,sys
sys.path.append('/Users/ajc/Core/PyScript')
import readdata as rd
import os
import geopy.distance

In [ ]:
mesh_resfile= '/Users/ajc/Core/simulations/production/streams/SIM_13/mesh_coords_10m_min100m/ats_vis_surface_mesh.h5'
mesh_hres = rd.get_mesh_coords(mesh_resfile)

In [ ]:
folder='EFPC'#'portage_river'

path_high_res = path='/Users/ajc/Core/simulations/production/streams/SIM_13/mesh_coords_10m_min100m'

name = '10m_min100m'
sim = 'mesh_coords_%s'%name 
path='/Users/ajc/Core/simulations/production/streams/SIM_13/%s'%sim

infile = [os.path.join(path,'ats_vis_surface_data.h5'), os.path.join(path,'ats_vis_surface_mesh.h5'),
          os.path.join(path_high_res,'ats_vis_surface_mesh.h5') ]


outfile='/Users/ajc/Core/PostProcessing/ReactiveTransport/2021/Q2/'

In [ ]:
# read non-boundary and boundary cells
cells_file = np.loadtxt('/Users/ajc/Core/PreProcessing/ReactiveTransport/data/%s/reach_cells/vector_cells-%s.dat'%(folder,name),usecols=0,delimiter=',')
cells_file = np.array(cells_file,dtype=int)
bccells_file = np.loadtxt('/Users/ajc/Core/PreProcessing/ReactiveTransport/data/%s/reach_cells/vector_cells-%s.dat'%(folder,name),usecols=1,delimiter=',')
bccells_file = np.array(bccells_file,dtype=int)


c_merge_file = np.loadtxt('/Users/ajc/Core/PreProcessing/ReactiveTransport/data/%s/centriods_merged/centriods_merged-%s.dat'%(folder,name),usecols=0,delimiter=',',dtype=str)
cent_merge = np.array(c_merge_file,dtype=str)

# Read discharge
Q_file = h5py.File('/Users/ajc/Core/PreProcessing/ReactiveTransport/data/%s/discharge/discharge-%s.h5'%(folder,name),'r')
discharge = np.concatenate(Q_file['surface-mass_flux.face.0']['0'][:])

#Read source
Src_file = h5py.File('/Users/ajc/Core/PreProcessing/ReactiveTransport/data/%s/implied_source/implied_watersource-%s.h5'%(folder,name),'r')
source = np.concatenate(Src_file['surface-water_mass_source.cell.0']['0'][:])



In [ ]:
def get_coarseres_cells(cells_file):
    csum = np.cumsum(cells_file)
    for c in csum:
        yield c
        
ptr = get_coarseres_cells(cells_file)
Q = []
xprev = 0
bc_count = 0
for i,p in enumerate(ptr):
    if bccells_file[i] == 0:
        Q.append(discharge[xprev+bc_count:p+bc_count])
        bc_count = bc_count + 1
        xprev = p 
    else:
        Q.append(discharge[xprev+bc_count:p+bc_count])
        xprev = p

Q =  np.concatenate(Q)

print (len(Q), len(source), len(discharge))

In [ ]:
mesh = rd.get_mesh_coords(infile[1])

In [ ]:
variables = [(0,'DOM1'), (1,'O2(aq)'), (2,'NO3-'), (3,'HCO3-'),(4,'H+'),(5,'N2(aq)')]
#var = 'DOM1 conc'
var_c = 0
var = variables[var_c][0]
C_data = rd.GetSurfVarFromVis_All(infile[0],'surface-total_component_concentration',suffix='.cell.%s'%var)
Concentration = C_data['surface-total_component_concentration']
cycles = len(Concentration)

In [ ]:

EFPC= True#False
def get_limits():
    fig, ax = plt.subplots(2,1,figsize=(6,8))#,subplot_kw={"projection": "3d"})
    #fig = plt.figure(figsize=(20,8))
    #ax = fig.add_subplot(121)

    ax[0].scatter(mesh['x'],mesh['y'],marker=".",c=mesh['z'],s=0.1)
    #ax.set_xlim(xmin-0.0005,xmax)
    xmin, xmax = ax[0].get_xlim()
    ymin, ymax = ax[0].get_ylim()
    #EFPC:
    xmin, ymin =-84.40676658316265, 35.92218408032078
    xmax, ymax = -84.23017702238597, 36.0329686150259
    print(xmin,xmax)
    print(ymin,ymax)
    # Coweeta
    #X0 = [-83.48, -83.40]
    #Y0 = [35.03, 35.08]
    
    if EFPC:
        #X0 = [-84.22, -84.405]
        #Y0 = [35.92, 36.04]
        X0 = [xmin, xmax]
        Y0 = [ymin, ymax]
    else:#Portage
        X0 = [-83.68, -83.40]
        Y0 = [41.02, 41.41]
    
    #ax[0].set_xlim(X0)
    #ax[0].set_ylim(Y0)
    #xmin, xmax = ax[0].get_xlim()
    #ymin, ymax = ax[0].get_ylim()

    xx = geopy.distance.geodesic(xmin,xmax).km #+ 0.07
    yy = geopy.distance.geodesic(ymin,ymax).km #+ 0.455
    print (xx,yy)
    Xlabels = np.round(np.linspace(0,xx,10),0)
    Ylabels = np.round(np.linspace(0,yy,8),0)
    Xlabels = [int(i) for i in Xlabels]
    Ylabels = [int(i) for i in Ylabels]
    print (Xlabels)
    #ax = fig.add_subplot(122)
    #ax[1].scatter(mesh['x'],mesh['y'],marker=".",c=mesh['z'])
    ax[1].scatter(mesh['x'],mesh['y'],marker=".",c=Q,s=.1,cmap=cm.coolwarm,vmin=1e-3,vmax=1,norm=matplotlib.colors.LogNorm())
    #print (len(X_mesh),len(Y_mesh),len(Z_mesh))
    #ax[1].plot_surface(X_mesh, Y_mesh, [Z_mesh,Z_mesh], cmap=cm.coolwarm,linewidth=0, antialiased=False)
    ax[1].set_xticklabels(Xlabels)
    ax[1].set_yticklabels(Ylabels)
    ax[1].set_xticklabels(Xlabels)
    ax[1].set_yticklabels(Ylabels)
    ax[1].set_xlabel('Distance [km]',fontsize=12,fontweight='normal')
    ax[1].set_ylabel('Distance [km]',fontsize=12,fontweight='normal')
    #ax[1].set_xlim(xmin,xmax)
    #ax[1].set_ylim(ymin,ymax)
    #plt.gca().invert_xaxis()
    plt.show()
    return X0, Y0, Xlabels, Ylabels
    
X0, Y0, Xlabels,Ylabels = get_limits()



In [ ]:
#%matplotlib qt

fig, ax = plt.subplots(1,1,figsize=(8,4))
#fig, ax = plt.subplots(2,2,figsize=(10,3),gridspec_kw={'width_ratios': [1, 1.2]})
centroid = []
thick = [1,1,1]
kk = -1#282 + 63 + 28 + 198 + 82 + 166 + 291 + 117 + 95 + 900 
for k in range(1):
    ran = Q[:kk]
    
    #sc1 = ax.scatter(mesh_hres['x'][:kk],mesh_hres['y'][:kk],marker='.',c='lightgrey',s=0.1)
    #ax.scatter(mesh['x'],mesh['y'],marker=".",c=mesh['z'],s=0.1)
    sc = ax.scatter(mesh['x'][:kk],mesh['y'][:kk],marker='.',c=ran,s=0.1,cmap=cm.coolwarm,vmin=1e-3,vmax=1,norm=matplotlib.colors.LogNorm())
    
    #sc = ax.scatter(X_mesh[:kk],Y_mesh[:kk],marker='.',c=source[:kk],s=20,cmap=cm.coolwarm,vmin=5e-4,vmax=1e-2,norm=matplotlib.colors.LogNorm())
    
    #sc = plt.plot_surface(X_mesh[:kk],Y_mesh[:kk],z=ran,linestyle='solid')#,s=6,plotnonfinite=True,cmap=cm.coolwarm,vmin=5e-4,vmax=1,norm=matplotlib.colors.LogNorm())
    
    cbar = plt.colorbar(sc, label='Discharge [m$^3$/sec]',shrink=0.8,aspect=30)
    cbar.set_label('Discharge [m$^3$/sec]',size=12)
    cbar.ax.tick_params(labelsize=10) 
    plt.tight_layout(h_pad=1)
    #ax[0].set_xlim(X0)
    
    #ax[0].set_ylim(Y0)
    #xmin, xmax = ax.get_xlim()
    #ymin, ymax = ax.get_ylim()
    #plt.gca().invert_xaxis()
    ax.set_xticklabels(Xlabels)
    ax.set_yticklabels(Ylabels)
    ax.set_xlabel('Distance [km]',fontsize=12,fontweight='normal')
    ax.set_ylabel('Distance [km]',fontsize=12,fontweight='normal')
    plt.tight_layout()
    plt.savefig('/Users/ajc/Core/PostProcessing/ReactiveTransport/2021/Q2/efpc-10m_merge.png',dpi=400)